In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.cluster import KMeans
from imblearn.under_sampling import EditedNearestNeighbours
import tensorflow as tf
from tensorflow.keras.optimizers import legacy
from tensorflow.keras import layers, Model
import pickle


2024-05-04 01:27:47.146943: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 01:27:47.224390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 01:27:47.224435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 01:27:47.227506: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-04 01:27:47.253943: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 01:27:47.256274: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:

df = pd.read_csv('pre-processed.csv')

In [3]:
print('Final: \n', df.iloc[:,-1].value_counts())

Final: 
 Benign                      13390249
DDOS attack-HOIC              686012
DDoS attacks-LOIC-HTTP        576191
DoS attacks-Hulk              461912
Bot                           286191
FTP-BruteForce                193354
SSH-Bruteforce                187589
Infilteration                 160639
DoS attacks-SlowHTTPTest      139890
DoS attacks-GoldenEye          41508
DoS attacks-Slowloris          10990
DDOS attack-LOIC-UDP            1730
Brute Force -Web                 611
Brute Force -XSS                 230
SQL Injection                     87
Name: Label, dtype: int64


In [4]:
# Buoc nay dung de loai bo cac class co nhieu data (du lieu lon), nham muc dich chay demo
l = []
for lb, cnt in df.iloc[:,-1].value_counts().items():
  if cnt > 2000:
    l.append(lb)

print(l)

['Benign', 'DDOS attack-HOIC', 'DDoS attacks-LOIC-HTTP', 'DoS attacks-Hulk', 'Bot', 'FTP-BruteForce', 'SSH-Bruteforce', 'Infilteration', 'DoS attacks-SlowHTTPTest', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris']


In [5]:
# Xác định cột nhãn là cột cuối cùng
label_column = df.columns[-1]

# Cập nhật df bằng cách loại bỏ các dòng có nhãn nằm trong danh sách lbs
df = df[~df[label_column].isin(l)]

print('Final: \n', df.iloc[:,-1].value_counts())

Final: 
 DDOS attack-LOIC-UDP    1730
Brute Force -Web         611
Brute Force -XSS         230
SQL Injection             87
Name: Label, dtype: int64


In [6]:
# Tách dữ liệu thành features (X) và label (y)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [7]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỉ lệ 7:3 theo các lớp
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [8]:
print("Kích thước của tập huấn luyện (X_train):", X_train.shape)
print("Kích thước của tập kiểm tra (X_test):", X_test.shape)
print("Kích thước của tập huấn luyện (y_train):", y_train.shape)
print("Kích thước của tập kiểm tra (y_test):", y_test.shape)

Kích thước của tập huấn luyện (X_train): (1860, 78)
Kích thước của tập kiểm tra (X_test): (798, 78)
Kích thước của tập huấn luyện (y_train): (1860,)
Kích thước của tập kiểm tra (y_test): (798,)


In [9]:
# Đếm số lượng mẫu của mỗi lớp trong tập huấn luyện (X_train)
class_counts_train = y_train.value_counts()

# Đếm số lượng mẫu của mỗi lớp trong tập kiểm tra (X_test)
class_counts_test = y_test.value_counts()

# In số lượng mẫu của mỗi lớp trong tập huấn luyện và tập kiểm tra
print("Số lượng mẫu của mỗi lớp trong tập huấn luyện:")
print(class_counts_train)
print("\nSố lượng mẫu của mỗi lớp trong tập kiểm tra:")
print(class_counts_test)

Số lượng mẫu của mỗi lớp trong tập huấn luyện:
DDOS attack-LOIC-UDP    1211
Brute Force -Web         427
Brute Force -XSS         161
SQL Injection             61
Name: Label, dtype: int64

Số lượng mẫu của mỗi lớp trong tập kiểm tra:
DDOS attack-LOIC-UDP    519
Brute Force -Web        184
Brute Force -XSS         69
SQL Injection            26
Name: Label, dtype: int64


In [10]:
t = 400 # Maximum number in a label
# Lay danh sach cac class thuoc minority va majority
Smin = []
Smaj = []

for label, cnt in class_counts_train.items():
  if cnt >= (t * 0.7): # trainset:testset = 7:3
    Smaj.append(label)
  else:
    Smin.append(label)

print(Smin)
print(Smaj)

['Brute Force -XSS', 'SQL Injection']
['DDOS attack-LOIC-UDP', 'Brute Force -Web']


In [11]:
# Số lượng cụm và tổng số mẫu cần lấy từ mỗi lớp
n_clusters = 4 # Num of classes
samples_needed_per_class = 280 # t * 0.7

# Khởi tạo một list để lưu các cụm và nhãn sau khi xử lý
selected_samples_X = []
selected_samples_y = []

# Lọc ra các mẫu dữ liệu chỉ thuộc các nhãn trong Smaj
majority_mask = np.isin(y_train, Smaj)
X_majority = X_train[majority_mask]
y_majority = y_train[majority_mask]

print('ENN')
# Áp dụng ENN để loại bỏ nhiễu
enn = EditedNearestNeighbours()
X_enn, y_enn = enn.fit_resample(X_majority, y_majority)

# Duyệt qua từng nhãn trong nhãn đã làm sạch
for label in Smaj:
    print('Label: ', label)
    label_mask = y_enn == label
    X_label = X_enn[label_mask]
    
    # Áp dụng k-means trên dữ liệu đã làm sạch cho mỗi nhãn
    if len(X_label) > 1:  # Kiểm tra có đủ dữ liệu để phân cụm không
        print('Clustering')
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans.fit(X_label)
        labels = kmeans.labels_

        # Tính số mẫu cần lấy từ mỗi cụm
        cluster_counts = np.bincount(labels, minlength=n_clusters)
        cluster_ratios = cluster_counts / cluster_counts.sum()
        samples_per_cluster = np.floor(cluster_ratios * samples_needed_per_class).astype(int)

        # Đảm bảo rằng tổng số mẫu đúng bằng 16000, điều chỉnh nếu cần
        while samples_per_cluster.sum() < samples_needed_per_class:
            print('not enough')
            samples_per_cluster[np.argmax(cluster_counts - samples_per_cluster)] += 1

        print('Compressing')
        # Lấy mẫu từ mỗi cụm
        for cluster in range(n_clusters):
            print('Cluster ', cluster)

            cluster_mask = labels == cluster
            X_cluster = X_label[cluster_mask]
            
            if len(X_cluster) > 0:

                # Chọn ngẫu nhiên các mẫu từ cụm
                if len(X_cluster) > samples_per_cluster[cluster]:
                    indices = np.random.choice(len(X_cluster), samples_per_cluster[cluster], replace=False)
                    selected_samples_X.append(X_cluster.iloc[indices])
                    selected_samples_y.extend([label] * samples_per_cluster[cluster])
                else:
                    selected_samples_X.append(X_cluster)
                    selected_samples_y.extend([label] * len(X_cluster))

# Tạo tập dữ liệu cuối cùng từ các mẫu đã chọn
X_train_balanced = np.vstack(selected_samples_X)
y_train_balanced = np.array(selected_samples_y)

print("Số lượng mẫu cuối cùng được chọn: ", X_train_balanced.shape[0])


ENN


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f36c1665550>
Traceback (most recent call last):
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: 

Label:  DDOS attack-LOIC-UDP
Clustering


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f373e27fc10>
Traceback (most recent call last):
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modul

not enough
not enough
Compressing
Cluster  0
Cluster  1
Cluster  2
Cluster  3
Label:  Brute Force -Web
Clustering


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f36bbd6dd30>
Traceback (most recent call last):
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modul

not enough
not enough
not enough
Compressing
Cluster  0
Cluster  1
Cluster  2
Cluster  3
Số lượng mẫu cuối cùng được chọn:  560


In [12]:
unique_values, counts = np.unique(y_train_balanced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

Brute Force -Web: 280
DDOS attack-LOIC-UDP: 280


In [13]:
# Khai báo các hằng số
BATCH_SIZE = 32
LATENT_DIM = 128
NUM_EPOCHS = 10
SAMPLES_TO_GENERATE = int(t * 0.7)

# Định nghĩa mô hình Generator
def build_generator():
    noise = layers.Input(shape=(LATENT_DIM,))
    x = layers.Dense(256, activation="relu")(noise)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(np.prod(X_train.shape[1:]), activation=None)(x)
    samp = layers.Reshape(X_train.shape[1:])(x)
    return Model(noise, samp)

# Định nghĩa mô hình Discriminator
def build_discriminator():
    samp = layers.Input(shape=X_train.shape[1:])
    x = layers.Flatten()(samp)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(1)(x)
    return Model(samp, x)


# Hàm để sinh dữ liệu
def generate_data_for_class(label):
    global X_train_balanced, y_train_balanced, X_train, y_train
    current_data = X_train[y_train == label]
    current_samples = current_data.shape[0]
    samples_needed = SAMPLES_TO_GENERATE - current_samples

    X_train_balanced = np.concatenate([X_train_balanced, current_data], axis=0)
    y_train_balanced = np.concatenate([y_train_balanced, np.full(current_samples, label)], axis=0)

    if samples_needed > 0:
        noise = np.random.normal(0, 1, (samples_needed, LATENT_DIM))
        generated_samples = generator.predict(noise)

        # Gộp dữ liệu sinh ra vào X_train và y_train
        X_train_balanced = np.concatenate([X_train_balanced, generated_samples], axis=0)
        y_train_balanced = np.concatenate([y_train_balanced, np.full(samples_needed, label)], axis=0)

# Huấn luyện WGAN cho mỗi class thiểu số
for label in Smin:

    # Tối ưu hoá và loss function
    optimizer = legacy.RMSprop(lr=0.00005)

    generator = build_generator()
    discriminator = build_discriminator()

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    discriminator.trainable = False

    noise = layers.Input(shape=(LATENT_DIM,))
    samp = generator(noise)
    validity = discriminator(samp)
    combined = Model(noise, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    data = X_train[y_train == label]
    valid = np.ones((BATCH_SIZE, 1))
    fake = -np.ones((BATCH_SIZE, 1))
    
    for epoch in range(NUM_EPOCHS):
        idx = np.random.randint(0, data.shape[0], BATCH_SIZE)
        real_samp = data.iloc[idx]

        noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))
        gen_samp = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_samp, valid)
        d_loss_fake = discriminator.train_on_batch(gen_samp, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = combined.train_on_batch(noise, valid)

    generate_data_for_class(label)  # Sinh dữ liệu cho class hiện tại



/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 0s 3ms/step


/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 0s 3ms/step


In [14]:
unique_values, counts = np.unique(y_train_balanced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

Brute Force -Web: 280
Brute Force -XSS: 280
DDOS attack-LOIC-UDP: 280
SQL Injection: 280


In [15]:
# Giam du lieu X_test, y_test bang cach chon ngau nhien

def reduce_data(X_test, y_test, n):
    unique_classes = np.unique(y_test)
    X_test_reduced = np.empty((0, X_test.shape[1]))
    y_test_reduced = np.array([])

    for label in unique_classes:
        indices = np.where(y_test == label)[0]
        if len(indices) > n:
            indices = np.random.choice(indices, n, replace=False)
        X_subset = X_test.iloc[indices]
        y_subset = y_test.iloc[indices]

        X_test_reduced = np.vstack((X_test_reduced, X_subset))
        y_test_reduced = np.concatenate((y_test_reduced, y_subset))

    return X_test_reduced, y_test_reduced

X_test_reduced, y_test_reduced = reduce_data(X_test, y_test, int(t * 0.3))


In [16]:
unique_values, counts = np.unique(y_test_reduced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

Brute Force -Web: 120
Brute Force -XSS: 69
DDOS attack-LOIC-UDP: 120
SQL Injection: 26


In [17]:
# Lưu dữ liệu vào file
data = {
    'X_train': X_train_balanced,
    'X_test': X_test_reduced,
    'y_train': y_train_balanced,
    'y_test': y_test_reduced
}

with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)
